<i><b>Public-AI</b></i>

### week 2. 협업 필터링
# Section 2. 협업 필터링 알고리즘의 종류 및 구현

협업 필터링에는 유저 기반 협업 필터링과 아이템 기반 협업 필터링이 있습니다. 두 가지 종류의 협업 필터링 알고리즘을 배우면서 협업 필터링을 보다 깊이 이해해 보겠습니다. 그 다음에는 앞서 샘플 리뷰 데이터로 배운 유사도 계산을 보다 큰 데이터셋에 적용해 아이템 기반 협업 필터링을 직접 구현해 보겠습니다.

### _Objective_ 

* [ **협업필터링 알고리즘의 종류** ] 협업 필터링의 핵심 알고리즘, 유저 기반 협업 필터링과 아이템 기반 협업 필터링의 개념을 배웁니다.
* [ **구현** ] Movie-lens 데이터를 이용해 아이템 기반 협업 필터링을 구현해봅니다.



In [ ]:
# 필요한 라이브러리 가져오기
%matplotlib inline

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tensorflow.keras.utils import get_file
np.set_printoptions(3)

#### 데이터 가져오기

이번 시간 동안 Pandas로 연산을 진행하기 위해서, 미리 CSV파일로 만들어진 데이터들을 직접 부르도록 하겠습니다.

In [ ]:
ROOT_URL = "https://pai-datasets.s3.ap-northeast-2.amazonaws.com/recommender_systems/movielens/datasets"

movie_path = get_file("movies.csv", os.path.join(ROOT_URL, "movies.csv"))
movie_df = pd.read_csv(movie_path)

genre_path = get_file("genres.csv", os.path.join(ROOT_URL, "genres.csv"))
genre_df = pd.read_csv(genre_path)

rating_path = get_file("ratings.csv", os.path.join(ROOT_URL, "ratings.csv"))
rating_df = pd.read_csv(rating_path)

# 메모리 문제를 좀 더 완화하기 위해 8비트로 변경(0~255)
rating_df.rating = (rating_df.rating*2).astype(np.uint8)
rating_df.movie_id = rating_df.movie_id.astype(np.uint32)
rating_df.user_id = rating_df.user_id.astype(np.uint32)

rating_df.drop(columns='rated_at',inplace=True)

# \[ 협업 필터링 알고리즘의 종류: 유저 기반 협업 필터링과 아이템 기반 협업 필터링 \]
---


## 1. User-Based Collaborative Filtering 과 Item-Based Collaborative Filtering

<img src="https://i.imgur.com/JRe39qe.png" width="500">

협업 필터링(Collaborative Filtering)을 통해 추천하는 방법은 크게 2가지가 있습니다. 

* 유저 기반 협업 필터링(User-Based Collaborative Filtering) : 유저와 비슷한 유저가 좋게 평가한 영화를 추천
* 아이템 기반 협업 필터링(Item-Based Collaborative Filtering) : 유저가 좋아하는 영화와 비슷한 영화를 추천

두 알고리즘의 중요한 차이는 바로 **무엇에 대한 유사도**를 계산하느냐에 있습니다. 유저 기반 협업 필터링은 유저 간의 유사도를 계산하여 추천하는 방식이고, 아이템 기반 협업 필터링은 아이템 간의 유사도를 계산하는 방식입니다. 즉, 앞서 Section 1에서 배운 유사도를 이용하여 유저 간의 유사도를 계산할 것인지, 제품 간의 유사도를 계산할 것인지에 따른 차이가 있는 것이죠. 

## 2. User Similarity Matrix

Section 1에서는 유저 5명을 놓고 유사도를 계산해보았는데, 실제 시스템에는 훨씬 많은 유저가 있습니다. 유저 간의 유사도 행렬(User Similarity Matrix)이란 아래와 같이 각 유저 간의 영화 평점을 가지고 유사도 점수를 계산해 넣은 행렬입니다.이전 시간에 했던 유사도 계산이 사실상 위의 행렬 내 값들을 찾는 것과 동일합니다.

<img src="https://i.imgur.com/GNPZj56.png" width="500">

## 3. Item Similarity Matrix
아이템 간의 유사도 행렬(Item Similarity Matrix)이란 역으로 각 영화 별 유저들의 레이팅에 대한 비교로 유사도 점수를 계산해 넣어 만든 행럴입니다. 아마존에서 사용하고 있는 추천시스템이 바로 이러한 아이템 유사도 행렬을 사용하여 **아이템 기반 협업 필터링(Item-Based Collaborative Filtering) 추천 시스템**입니다. 주기적으로 아이템 유사도 행렬을 계산해주기만 하면, 운영 환경에서는 계산된 유사도 행렬에 따라 제품을 추천하면 되기 때문에 서버의 연산량 이슈가 크게 발생하지 않는 효율적인 추천 시스템입니다. 이번 시간에는 아래의 아이템 기반 협업필터링(Item-Based Collaborative Filtering)을 중심으로 구현해보도록 하겠습니다.

<img src="https://i.imgur.com/UaOmANU.png" width="500">

# \[ 구현 \]
---

Movie-lens 데이터를 이용해 아이템 기반 협업 필터링을 구현해보겠습니다. 보다 구체적으로는 전체 데이터를 사용할 경우 데이터가 보이는 희소성 문제를 살펴본 후, 이 문제점을 해결하기 위해 기준을 세워 협업 필터링 시스템에서 사용할 아이템과 유저 데이터를 걸러내겠습니다. 그 다음에 지난 section에서 배운 유사도 알고리즘으로 유저도 행렬을 만들어보겠습니다. 끝으로는 유사도 행렬을 이용해 영화를 추천해보겠습니다.

## 1. 데이터 속 문제 - 데이터의 Sparsity

효과적인 추천시스템을 만들기 위해 가지고 있는 모든 데이터를 써야할 것 같지만, 사실 가지고 있는 모든 데이터를 사용하게 되면 다양한 문제가 생깁니다. 데이터의 희소성(sparsity) 문제는 그 중 하나입니다. 데이터가 희소(Sparse)하다는 의미는 유저 수와 영화 수는 많은 데 비해서 유저와 아이템 사이의 상호작용 데이터는 극히 드물다는 것을 의미합니다. 한 유저가 평점을 남기는 영화는 전체 영화 중에 극히 일부일 것입니다. Movie-lens 데이터의 희소성은 어느 정도인지 알아봅시다.

유저와 영화 정보를 가지고 유저-아이템 행렬(User-Item Matrix)를 만들면 30억개의 칸이 있는 행렬이 만들어지지만, 이 행렬을 채우는 평점 데이터는 2천만개 정도로, 전체의 0.5% 정도만 채워집니다.

## 2. 영화 최소 기준 선정하기

전체 리뷰 데이터를 이용해 만든 유저-아이템 행렬은 밀집도(Density)가 매우 낮습니다. 평가를 한 횟수가 매우 적은 사람이나 받은 평점이 매우 적은 영화의 경우에는 유저나 영화 간 유사도가 지나치게 높게 나오거나, 적게 나올 우려도 있습니다. 이를 방지하기 위해 적게 별점을 남긴 사람과 별점을 적게 받은 영화의 케이스를 제거해야 합니다. 

#### 영화 별 평점(rating)의 갯수 세기

#### 영화 별 평점(rating) 갯수의 누적합 비율 구하기

#### 전체 평점 데이터의 90% 수준에서의 최소 별점 수 
영화별 평점 데이터를 평점이 많은 순으로 합쳤을 때, 전체 평점 데이터의 90%쯤 되는 지점의 영화가 가지는 최소 별점 갯수를 알아보겠습니다.

#### 최소 별점 수를 보장하는 영화 리스트 가져오기

#### 기준에 부합하는 영화만 포함하는 채로 두기

In [ ]:
truncated_rating_df = "" # TODO 

## 3. 유저 최소 기준 선정하기
이번에는 동일한 방법으로 유사도 계산에 사용할 유저를 정할 최소 기준을 정해봅시다.

#### 유저 별 평점(rating)의 갯수 세기

#### 유저 별 평점(rating) 갯수의 누적합 비율 구하기

#### 전체 평점 데이터의 90% 수준에서의 최소 별점 수 
유저 별 평점 데이터를 평점이 많은 순으로 합쳤을 때, 전체 평점 데이터의 90%쯤 되는 지점의 유저가 가지는 최소 별점 갯수를 알아보겠습니다.

#### 최소 별점 수를 보장하는 유저 리스트 가져오기

#### 기준에 부합하는 유저만 포함하는 채로 두기

In [ ]:
truncated_rating_df = "" # TODO 

## 4. User-Item Matrix 만들기

리뷰 데이터의 정보를 최대한 유지하면서도, 유저-아이템 행렬의 희소성 문제를 줄일 수 있도록 유저와 영화를 적정 수준에서 걸러낸 `truncated_rating_df`를 만들었습니다. 이를 활용해 유저-아이템 행렬을 만들어봅시다. 데이터가 커서 만들어지는 데 다소 시간이 걸릴 수 있습니다.

In [ ]:
ui_matrix = (
    truncated_rating_df
    .pivot_table(
        values='rating',
        index='user_id',
        columns='movie_id'))

### (1) 영화의 편향과 유저의 편향을 제거하기

평점 데이터는 숫자 그대로 해석하게 되면 오류가 발생할 수 있습니다.
같은 4점이라고 해도 어떤 유저는 4점이면 최고의 극찬을 한 것일 수도 있고, 어떤 유저는 보통 4점은 주기 때문에 그다지 좋게 평가한 것이 아닐 수도 있기 때문입니다. 영화도 마찬가지로 영화 자체의 완성도로 인해 점수를 높게 받은 경우가 있고 적게 받은 경우가 있습니다. 고객의 성향을 잘 드러내기 위해서는 유저나 영화에 따른 편향을 제거해야 합니다. 

In [ ]:
avg_rating = ""

# 고객의 편향 계산하기
user_bias = ""
# 아이템의 편향 계산하기
item_bias = ""

# 편향이 제거된 정보
adjusted_ui_df = ""

#### 결측치를 0으로 채우기

결측치(Missing Value)를 어떻게 채워주어야 하는가는 매우 중요한 문제이고, 값을 잘못 채울 경우에는 큰 성능 저하가 생기기도 합니다. 지금은 가장 간단한 방식인 0으로 채우는 방식으로 진행합니다.

In [ ]:
adjusted_ui_df # 

----

#### \[ additional \]

항상 두 편향을 모두 제거해야만 하는 것은 아닙니다. 이후 우리가 어떤 수식을 기준으로 편향을 적용함에 따라서 편향을 제거하는 방법은 달라질 수 있습니다.

* Case 1) Pearon-Based Similarity

    아이템 편향만을 제거해주면, Pearson-Based Similarity를 계산하기 위한 보정 수식이 됩니다.

$$
sim(i,j) = \frac{\sum_{u \in U} (R_{u,i} - \bar{R_{i}})(R_{u,j}- \bar{R}_j)}{\sqrt{\sum_{u \in U} (R_{u,i} - \bar{R_{i}})^2}{\sqrt{\sum_{u \in U} (R_{u,j} - \bar{R_{j}})^2}} }
$$

* Case 2) Adjusted cosine Similarity
    
    유저 편향만을 제거해주면, Adjusted-Based Similarity를 계산하기 위한 보정 수식이 됩니다.
$$
sim(i,j) = \frac{\sum_{u \in U} (R_{u,i} - \bar{R_{u}})(R_{u,j}- \bar{R}_u)}{\sqrt{\sum_{u \in U} (R_{u,i} - \bar{R_{u}})^2}{\sqrt{\sum_{u \in U} (R_{u,j} - \bar{R_{u}})^2}} }
$$

----

## 5. 코사인 유사도를 활용하여 유사도 행렬 구성하기


앞서 배운 코사인 유사도 수식을 떠올려봅시다.
$$
Cosine\_Similarity(X,Y) =   \frac{X \cdot Y}{|X| |Y|}
$$

영화X와 영화Y의 코사인 유사도를 구한다고 하면, 각 영화에 대해 유저$1$부터 유저$n$번째 유저가 남긴 평점 벡터를 가져와 위의 연산을 수행해야 합니다. 이를 함수로 정의하면 아래와 같이 정의할 수 있을 것입니다. `calculate_cosine_similarity`의 입력값 `x`와 `y`는 각각 영화X와 영화Y에 대해 전체 유저가 남긴 평점 벡터입니다.

In [ ]:
def calculate_cosine_similarity(x, y):
    return #

In [ ]:
# Dataframe to array
adjusted_ui_matrix = adjusted_ui_df.values

# 빈행렬 생성
_, num_movies = adjusted_ui_df.shape
item_sim_matrix = np.zeros((num_movies, num_movies)) 

for i in range(num_movies):
    for j in range(num_movies):
        # Cosine Similarity 계산
        item_sim_matrix[i,j] = "" # TODO


### \[ Additional \]  위의 코드를 개선하기( Vectorize 이용하기 )

하지만, 위와 같이(For 구문) 코드를 작성할 경우 행렬을 구성하는 데에 굉장히 오랜 시간이 소요됩니다. 이를 보다 빠르게 하기 위해서는 numpy의 벡터 연산을 이용해 계산을 수행해야 합니다. 다시 코사인 유사도 수식을 조금 정리해봅시다. 
<br>
<br>
$$
Cosine\_Similarity(X,Y) = \frac{\sum_{i=1}^{n}x_i y_i}{\sqrt{\sum_{i=1}^{n}x_i^2} \sqrt{\sum_{i=1}^{n}y_i^2}} =  \frac{X \cdot Y}{|X| |Y|}
$$
<br>
코사인 유사도에서 분자에 해당하는 $\sum_{i=1}^{n}x_i y_i$는 사실 행렬$X$와 행렬$Y$에 행렬곱을 수행하는 것과 같고, 각 행렬 원소의 제곱합에 루트를 씌우는 것은 각 행렬의 크기(노름)를 구하는 것과 같습니다. 
<br>
<br>
$$
Cosine\_Similarity(X,Y) =  \frac{X \cdot Y}{|X| |Y|} = \frac{X}{|X|}\cdot\frac{Y}{|Y|} = \hat X \cdot \hat Y
$$
<br>
또한 이는 각 벡터를 단위벡터화(정규화)한 후, 행렬곱을 적용하는 것과 동일합니다. 이렇게 수식을 정리하면 훨씬 더 빠르게 코사인 유사도를 계산할 수 있습니다.행렬곱을 적용한 코사인 유사도 수식을 유저-아이템 행렬(`adjusted_ui_matrix`)에 적용하려면 어떻게 해야 할까요?<br>

<img src="https://i.imgur.com/e1Dpxnf.jpg" width="700">

아래와 같이 유저-아이템 행렬($R$)을 전치하여 만든 아이템-유저 행렬($R^{T}$) 를 각각 정규화한 후, 행렬곱을 수행해야 합니다($=\hat {R^T} \cdot \hat R$) <br>
혹은 유저-아이템 행렬($R$)을 정규화한 후 전치하여, 행렬곱을 수행해야 합니다($={\hat R}^T \cdot \hat R$) 
<br>
$$
Cosine\_Similarity(R^T,R) =  \frac{R^T \cdot R}{|R^T| |R|} = \frac{R^T}{|R^T|}\cdot\frac{R}{|R|} = \hat {R^T} \cdot \hat R = {\hat R}^T \cdot \hat R
$$
<br>

#### 1. 정규화하기

#### 2. 행렬곱연산 수행하기 

행렬곱을 이용하지 않았을 때에는 대략 1시간 정도 걸리는데, 행렬곱 연산을 이용하면 12.5초에 끝납니다. 대략 200배에서 300배 정도 빠르게 연산이 가능합니다.

---

### (1) 위의 코드를 개선하기 - `scikit-learn ` 모듈 이용하기

Scipy에서는 거리를 계산하는 것을 빠르게 해주는 코드들을 제공합니다.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

item_similarity_matrix = # TODO

### (2) 대각행렬을 0으로 채우기

대각행렬은 자기 자신과의 유사도를 계산하는 것이기 때문에 항상 1로 나옵니다. 이럴 경우, 자기 자신을 추천하는 문제가 발생할 수 있어 자기 자신에 대해서는 유사도를 0으로 바꾸어 주었습니다.

In [ ]:
# TODO

### (3) id를 title로 변경하기

movie_id는 우리가 읽기가 어렵기 때문에, movie_title로 변경해주도록 하겠습니다.

In [ ]:
movie_ids = ui_matrix.columns
id2title = dict(zip(movie_df.id,movie_df.title))
movie_titles = [id2title[i] for i in movie_ids]

cosine_similarity_df = pd.DataFrame(
    item_similarity_matrix, 
    index=movie_titles, 
    columns=movie_titles)

cosine_similarity_df

## 6. 유사도 행렬을 바탕으로 유사 제품 추천하기

이제 코사인 유사도로 채워진 유사도 행렬이 완성되었습니다. 유사도 행렬을 이용해서 어떤 영화를 보았다고 할 때, 어떤 영화를 추천하는지 봅시다.

#### 토이스토리와 유사한 영화

In [ ]:
(
    cosine_similarity_df
    .loc['Toy Story']
    .sort_values(ascending=False)
    .iloc[:5]
)

#### 스타워즈와 유사한 영화

In [ ]:
(
    cosine_similarity_df
    .loc['Star Wars: Episode IV - A New Hope']
    .sort_values(ascending=False)
    .iloc[:5]
)

#### 쏘우와 유사한 영화

In [ ]:
(
    cosine_similarity_df
    .loc['Saw']
    .sort_values(ascending=False)
    .iloc[:5]
)

#### 링와 유사한 영화

In [ ]:
(
    cosine_similarity_df
    .loc['Ring, The']
    .sort_values(ascending=False)
    .iloc[:5]
)

# \[  더 나아가기 \]
---



## 1. 유클리디안 유사도 수식을 통해 유사도 행렬 구성하기

앞서 코사인 유사도를 이용해 유사도 행렬을 구하고, 영화를 추천하는 시스템을 구현해보았습니다. 이번에는 코사인 유사도 대신 유클리디안 유사도를 써봅시다.


$$
distance(X,Y) = \sqrt{\sum(X_i - Y_i)^2} = \sqrt{\sum(X_i^2 + Y_i^2 - 2X_iY_i)} = \sqrt{\sum X_i^2 + \sum Y_i^2 - 2 X \cdot Y}
$$

이번에도 코사인 유사도 때와 같이 sklearn과 벡터 연산을 이용해 풀도록 하겠습니다.

### (1) scikit learn으로 풀기

In [ ]:
from sklearn.metrics.pairwise import euclidean_distances

dist = euclidean_distances(adjusted_ui_matrix.T)
euclidean_similarity_output = 1 / (1 + dist)

----

#### \[ Optional \] Vectorize로 연산하기

$$
distance(X,Y) = \sqrt{\sum(X_i - Y_i)^2} = \sqrt{\sum(X_i^2 + Y_i^2 - 2X_iY_i)} = \sqrt{\sum X_i^2 + \sum Y_i^2 - 2 X \cdot Y} \\
sim(X,Y) = \frac{1}{1+distance(X,Y)}
$$



#### 1. 제곱 값 계산하기  ($\sum X_i^2$,   $\sum Y_i^2$)

In [ ]:
square_R = np.sum(adjusted_ui_matrix**2, axis=0)

#### 2. Dot 연산 계산하기

In [ ]:
dot_R = np.dot(adjusted_ui_matrix.T, adjusted_ui_matrix)

#### 3. 유클리디안 거리 구하기

In [ ]:
euclidean_distance = np.sqrt(np.add.outer(square_R, square_R) - 2 * dot_R)

#### 4. 유클리디안 유사도 계산하기

In [ ]:
euclidean_similarity_output = 1/ (1 + euclidean_distance)

----

### (2) 대각행렬을 0으로 채우기


In [ ]:
np.fill_diagonal(euclidean_similarity_output, 0.)

### (3)   id를 title로 변경하기

movie_id는 우리가 읽기가 어렵기 때문에, movie_title로 변경해주도록 하겠습니다.

In [ ]:
movie_ids = ui_matrix.columns
id2title = dict(zip(movie_df.id,movie_df.title))
movie_titles = [id2title[i] for i in movie_ids]

euclidean_similarity_df = pd.DataFrame(
    euclidean_similarity_output, 
    index=movie_titles, 
    columns=movie_titles)

## 2. 유사도 행렬을 통해 유사 제품 추천하기
코사인 유사도를 이용한 추천 시스템과 어떤 차이가 있을까요? 코사인 유사도를 사용한 시스템에서는 비슷하면서도 다양한 영화가 추천된 반면, 유클리디안 거리 유사도를 이용한 추천 시스템에서는 각 시리즈의 후속작이 추천되는 결과가 나왔습니다. 시리즈물과 같이 매우 유사한 것을 추천하기 위해서는 유클리디안 거리 유사도가 좀 더 유용하고, 좀 더 다양한 영화를 추천하기 위해서는 코사인 유사도가 더 적합할 것입니다.

#### 토이스토리와 유사한 영화

In [ ]:
(
    euclidean_similarity_df
    .loc["Toy Story"]
    .sort_values(ascending=False)
    .iloc[:5]
)

#### 스타워즈와 유사한 영화

In [ ]:
(
    euclidean_similarity_df
    .loc['Star Wars: Episode IV - A New Hope']
    .sort_values(ascending=False)
    .iloc[:5]
)

#### 쏘우와 유사한 영화

In [ ]:
(
    euclidean_similarity_df
    .loc["Saw"]
    .sort_values(ascending=False)
    .iloc[:5]
)

#### 링와 유사한 영화

In [ ]:
(
    euclidean_similarity_df
    .loc['Ring, The']
    .sort_values(ascending=False)
    .iloc[:5]
)

#  

---

    Copyright(c) 2019 by Public AI. All rights reserved.
    Writen by PAI, SangJae Kang ( rocketgrowthsj@publicai.co.kr )  last updated on 2020/01/13


---